# Differential Privacy Demonstration

This notebook illustrates **two** core differential privacy (DP) techniques:

- **Laplace Mechanism (Global DP)**: Adding noise to a counting query.
- **Randomized Response (Local DP)**: Users flip their own bits with a certain probability.

We'll walk through these steps:
1. Create a synthetic binary dataset.
2. Use the **Laplace mechanism** to release a noisy count.
3. Apply **randomized response** to each individual's bit.
4. Demonstrate how we can estimate the true fraction of "Yes" answers.
5. Discuss a high-level note on **privacy budgets**.


## Step 1: Create a Synthetic Binary Dataset

We'll simulate 1,000 individuals, each with a binary feature (e.g., "Did they purchase our product?").
- `1` = Yes
- `0` = No

We assume ~30% say "Yes". We'll print the first 10 rows and the total count of "Yes" answers.

In [ ]:
import numpy as np
import pandas as pd
import random

np.random.seed(42)
N = 1000
data = np.random.choice([0,1], size=N, p=[0.7,0.3])  # ~30% 'Yes'
df = pd.DataFrame({'Purchased': data})

print("First 10 rows of data:\n", df.head(10))
print("\nTrue count of 'Yes' in entire dataset:", df['Purchased'].sum())


## Step 2: Laplace Mechanism (Global DP)

For a **count** query, the (global) **sensitivity** is `1` (changing one record changes the count by at most 1). The Laplace mechanism adds noise drawn from a Laplace distribution centered at 0 with scale `sensitivity / ε`.

We'll define a function to apply Laplace noise to our true count, then see how smaller ε yields more noise (and thus more privacy, but less accuracy).

In [ ]:
def laplace_mechanism(true_value, epsilon, sensitivity=1.0):
    scale = sensitivity / epsilon
    noise = np.random.laplace(0, scale, 1)[0]
    return true_value + noise

def dp_count_laplace(data, epsilon):
    true_count = np.sum(data)
    dp_result = laplace_mechanism(true_count, epsilon, sensitivity=1.0)
    return dp_result

for eps in [0.1, 0.5, 1.0, 2.0]:
    dp_count = dp_count_laplace(data, eps)
    print(f"DP Count with ε={eps}: {dp_count:.2f} (true count={sum(data)})")

print("\nObservation:")
print("Lower ε => bigger noise => better privacy, but less accurate results.")


## Step 3: Randomized Response (Local DP)

In **local differential privacy**, each user perturbs their own data **before** sending it to the aggregator. Here, each bit is flipped with probability `p`. This provides plausible deniability for any individual's true response.

We can then **estimate** the true fraction of `1`s by correcting for this known probability of flipping.

In [ ]:
def randomized_response_localDP(bit, p=0.25):
    flip = np.random.rand() < p
    if flip:
        return 1 - bit
    else:
        return bit

def local_dp_estimate(bits, p=0.25):
    noisy_bits = [randomized_response_localDP(b, p) for b in bits]
    observed_fraction = np.mean(noisy_bits)
    # Derive the true fraction from the observed fraction:
    # E[Reported] = (1-p)*true_fraction + p*(1 - true_fraction)
    # => true_fraction = (observed_fraction - p/2) / (1 - p)
    true_fraction_est = (observed_fraction - (p/2)) / (1 - p)
    return observed_fraction, true_fraction_est

true_fraction = np.mean(data)
observed_fr, est_fraction = local_dp_estimate(data, p=0.30)

print("\nLocal DP (Randomized Response) demonstration:")
print(f" True fraction of 'Yes': {true_fraction:.3f}")
print(f" Observed fraction in noisy data: {observed_fr:.3f}")
print(f" Estimated fraction after correction: {est_fraction:.3f}")


## Step 4: Privacy Budget (High-Level Note)

When performing **multiple queries** on the same dataset, you must track how their privacy costs **compose**. For the Laplace mechanism, the total ε can add up across queries. More sophisticated composition theorems (advanced, zero-Concentrated DP, or Rényi DP) give tighter bounds.

Randomized response also has a local privacy parameter that must be tuned to balance utility (accuracy) and individual privacy.

Note: For multiple queries on the same dataset, track your total privacy budget (ε).